# SCD type 1

In [0]:
create table datamodelling_lak.default.scd_type1(
  prod_id int,
  prod_name string,
  prod_cat string,
  processed_date date
)

In [0]:
insert into table datamodelling_lak.default.scd_type1
values
(1,"prod1","cat1",current_date()),
(2,"prod2","cat2",current_date()),
(3,"prod3","cat3",current_date())

In [0]:
create table datamodelling_lak.gold.scd_type1(
  prod_id int,
  prod_name string,
  prod_cat string,
  processed_date date
)

In [0]:
drop table src

In [0]:
%python
spark.sql(f"""select * from datamodelling_lak.default.scd_type1""").createOrReplaceTempView("src")

In [0]:
select * from src;

In [0]:
merge into datamodelling_lak.gold.scd_type1 as t
using src as s
on s.prod_id = t.prod_id
when matched and s.processed_date >= t.processed_date then update set *
when not matched then insert *

In [0]:
select * from datamodelling_lak.gold.scd_type1

In [0]:
update datamodelling_lak.default.scd_type1 set prod_cat = "new category"
where prod_id = 3

# SCD Type 2

In [0]:
create table datamodelling_lak.default.scd_type2(
  prod_id int,
  prod_name string,
  prod_cat string,
  processed_date date
)

In [0]:
insert into table datamodelling_lak.default.scd_type2
values
(1,"prod1","cat1",current_date()),
(2,"prod2","cat2",current_date()),
(3,"prod3","cat3",current_date())

In [0]:
select *,
      current_timestamp() as start_date,
      cast("3000-01-01" as timestamp) as end_date,
      "y" as status
from datamodelling_lak.default.scd_type2


In [0]:
%python
spark.sql(f""" select *,
          current_timestamp() as start_date,
          cast("3000-01-01" as timestamp) as end_date,
          "y" as is_current
from datamodelling_lak.default.scd_type2 """).createOrReplaceTempView("src2")


In [0]:
select * from src2

In [0]:
create or replace table datamodelling_lak.gold.scd_type2(
  prod_id int,
  prod_name string,
  prod_cat string,
  processed_date date,
  start_date date,
  end_date date,
  is_current string
)

In [0]:
merge into datamodelling_lak.gold.scd_type2 as t
using src2 as s
on s.prod_id = t.prod_id
and t.is_current = "y"
when matched and (
s.prod_name <> t.prod_name or
s.prod_cat <> t.prod_cat or 
s.processed_date <> t.processed_date
)
then 
update set 
  t.end_date = current_timestamp(),
  t.is_current = "n"





In [0]:
select * from datamodelling_lak.gold.scd_type2

In [0]:
merge into datamodelling_lak.gold.scd_type2 as t
using src2 as s
on s.prod_id = t.prod_id
and t.is_current = "y"
when not matched then insert(
  prod_id,
  prod_name,
  prod_cat,
  processed_date,
  start_date,
  end_date,
  is_current
) values(
  s.prod_id,
  s.prod_name,
  s.prod_cat,
  s.processed_date,
  s.start_date,
  s.end_date,
  s.is_current
)

In [0]:
select * from datamodelling_lak.gold.scd_type2

In [0]:
update datamodelling_lak.default.scd_type2 set prod_cat = "new category"
where prod_id = 3